In [57]:
import pandas as pd

import hvplot as hv
import hvplot.pandas
import holoviews as hv
from holoviews import opts# noqa

In [58]:
data = pd.read_csv('a-Liste des adhérents_cleaned.csv')

In [59]:
data.head()

,Métropole,Date d'adhésion,Prénom,NOM,nb Adhérents,Nom des membres famille,Adresse mail,Lieu de résidence,Téléphone,site internet,...,préparation et tenue de stand,élaboration de supports de communication,veille sur l'actualité du vélo,amélioration de la réglementation,atelier de réparation,cartographie collaborative,"autre, précisez.2",Unnamed: 41,Unnamed: 42,Vos attentes
0,x,21/03/2014,Manuel,LEGER,1.0,NaN,leger_manuel@yahoo.fr,Nouméa,285323; 962100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,x,21/03/2014,Emmanuelle,SAILLY,1.0,NaN,NaN,Nouméa,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,29/03/2014,Luc,MICHEL,1.0,NaN,mail supprimé à la demande de Luc Michel,Nouméa,748207,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voies protégées. Vélos électriques à prix rais...
3,NaN,29/03/2014,Michèle,JOHN,1.0,NaN,michele.c.john@gmail.com,Nouméa,991990,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,29/03/2014,Daniel,BRETAUDEAU,1.0,NaN,danielbretaudeau@gmail.com,NaN,775322,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
data["date_adhesion"] = pd.to_datetime(data["Date d'adhésion"],format='%d/%m/%Y')


In [61]:
data.loc[data['nb Adhérents'] > 1, "montant_adhesion"] = '4000'
data.loc[data['nb Adhérents'] == 1, "montant_adhesion"] = '2000'

data.head()

,Métropole,Date d'adhésion,Prénom,NOM,nb Adhérents,Nom des membres famille,Adresse mail,Lieu de résidence,Téléphone,site internet,...,veille sur l'actualité du vélo,amélioration de la réglementation,atelier de réparation,cartographie collaborative,"autre, précisez.2",Unnamed: 41,Unnamed: 42,Vos attentes,date_adhesion,montant_adhesion
0,x,21/03/2014,Manuel,LEGER,1.0,NaN,leger_manuel@yahoo.fr,Nouméa,285323; 962100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-21,2000
1,x,21/03/2014,Emmanuelle,SAILLY,1.0,NaN,NaN,Nouméa,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-21,2000
2,NaN,29/03/2014,Luc,MICHEL,1.0,NaN,mail supprimé à la demande de Luc Michel,Nouméa,748207,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voies protégées. Vélos électriques à prix rais...,2014-03-29,2000
3,NaN,29/03/2014,Michèle,JOHN,1.0,NaN,michele.c.john@gmail.com,Nouméa,991990,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-29,2000
4,NaN,29/03/2014,Daniel,BRETAUDEAU,1.0,NaN,danielbretaudeau@gmail.com,NaN,775322,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-29,2000


In [62]:
adherent_2020 = data["date_adhesion"] >= "2019"
d_2020 = data[adherent_2020]

In [63]:
famille_2020 = d_2020.loc[(d_2020["nb Adhérents"]>1)]

In [64]:
d_2020

,Métropole,Date d'adhésion,Prénom,NOM,nb Adhérents,Nom des membres famille,Adresse mail,Lieu de résidence,Téléphone,site internet,...,veille sur l'actualité du vélo,amélioration de la réglementation,atelier de réparation,cartographie collaborative,"autre, précisez.2",Unnamed: 41,Unnamed: 42,Vos attentes,date_adhesion,montant_adhesion
154,NaN,23/02/2019,Yvonnick,PENDUFF,1.0,Martine BOULANGER,mpenduff@mls.nc,Nouméa,259952; 900259; 944368,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-02-23,2000
155,NaN,01/03/2019,Noele,HIRIGARAY,1.0,NaN,noele_hirigaray@hotmail.com,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-01,2000
156,NaN,25/03/2019,Claire,DRUYLANS,1.0,NaN,acdc@mls.nc,Nouméa,935984,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-25,2000
157,NaN,11/04/2019,Rodolphe,BERTOLINI,1.0,NaN,NaN,Nouméa,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-11,2000
158,NaN,11/04/2019,Raymond,CONWAY,4.0,NaN,NaN,Nouméa,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-11,4000
159,NaN,11/04/2019,Arnaud,DERICQ,2.0,Maël (6 ans),NaN,Nouméa,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-11,4000
160,NaN,11/04/2019,Remi,DODEMONT,1.0,NaN,garriguedodemont@gmail.com,Nouméa,241634,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-11,2000
161,NaN,11/04/2019,Régis,ETAIX-BONNIN,3.0,NaN,retaix@yahoo.fr,Nouméa,254213,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-11,4000
162,NaN,11/04/2019,Evelyne,FOLLIARD,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-11,2000
163,NaN,11/04/2019,Dominique,GARNIER,1.0,"Dominique, Laura, Marceau",dgarnier@canl.nc,Nouméa,851548,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-11,2000


In [140]:
individuel_2020 = d_2020.loc[(d_2020["nb Adhérents"]==1)]
adhesion_indiv = len(individuel_2020)
adhesion_famille = len(famille_2020)
nb_adhesion = adhesion_famille + adhesion_indiv
montant_tresor = adhesion_famille * 4000 +adhesion_indiv *2000
nbAdherent_famille =famille_2020["nb Adhérents"].sum()
nbAdherent = len(individuel_2020) + nbAdherent_famille
famille_2020_sorted = famille_2020.sort_values(by=["nb Adhérents"])
df_type_famille = famille_2020_sorted.groupby(['nb Adhérents'])
bar = df_type_famille.size().hvplot.bar().opts(title="Nombre de famille selon le nombre d'adhérent",ylabel="nb famille")
moyenne = hv.VLine(famille_2020["nb Adhérents"].mean()-1)
layout = bar * moyenne
layout.opts(opts.VLine(color='red', line_width=4))

:Overlay
   .Bars.A_0 :Bars   [nb Adhérents]   (0)
   .VLine.I  :VLine   [x,y]

In [66]:
dfm = pd.DataFrame({'nb_adherents': range(1,200)})

In [67]:
dfm['tresor_4000'] = dfm['nb_adherents']*4000

In [68]:
dfm['tresor_3000'] = dfm['nb_adherents']*3000

In [69]:
dfm['tresor_2000'] = dfm['nb_adherents']*2000

In [142]:
def formatter(value):
    return str(value) 

In [143]:
nbAdherent_famille

62.0

In [146]:
nbAdherent_line = hv.VLine(nbAdherent).opts(alpha=0.5,color='grey', line_width=2,line_dash='dashed' )

famille_points = hv.Points((adhesion_famille,adhesion_famille*4000)).opts( size=8)
indiv_point = hv.Points((adhesion_indiv,adhesion_indiv*2000)).opts(color='green', size=8)
famille_adherents_points = hv.Points((nbAdherent_famille,adhesion_famille*4000)).opts(size=8)

total_point = hv.Points((nb_adhesion,montant_tresor)).opts(color='grey', size=8)

famille_Hline = hv.HLine(adhesion_famille*4000).opts(alpha=0.3,color='blue',line_width=1)


layout = (dfm.hvplot.line(x="nb_adherents", y="tresor_4000") *
 dfm.hvplot.line(x="nb_adherents", y="tresor_3000") * 
 dfm.hvplot.line(x="nb_adherents", y="tresor_2000") * 
nbAdherent_line *
          famille_points *
          indiv_point * 
          total_point* 
          famille_adherents_points *
         famille_Hline).opts(xlabel="Nb adhésions", ylabel="Trésor",yformatter=formatter,ylim=(0,500000),xlim=(0,150),show_grid=True)
layout


:Overlay
   .Curve.I    :Curve   [nb_adherents]   (tresor_4000)
   .Curve.II   :Curve   [nb_adherents]   (tresor_3000)
   .Curve.III  :Curve   [nb_adherents]   (tresor_2000)
   .VLine.I    :VLine   [x,y]
   .Points.I   :Points   [x,y]
   .Points.II  :Points   [x,y]
   .Points.III :Points   [x,y]
   .Points.IV  :Points   [x,y]
   .HLine.I    :HLine   [x,y]

In [72]:

famille_2020.hvplot.line(x='date_adhesion', y='nb Adhérents')

:Curve   [date_adhesion]   (nb Adhérents)